In [83]:
import numpy as np 
import pandas as pd 
import os
import torch
import torchvision
import torch.nn as nn 
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
import torch.nn.functional as F 
from torch.utils.data import DataLoader, Dataset 
from torchvision.datasets import ImageFolder
import torch.optim as optim 
from PIL import Image
from tqdm import tqdm
from torchvision import models

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.to(device)
checkpoint = torch.load("models/checkpoint_epoch_resnet18_1000.pt", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"]) 
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.eval()

/home/matthew/anaconda3/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/matthew/anaconda3/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [85]:
dataset_test = ImageFolder("test_set/", 
                     transform=transforms.Compose([
                         transforms.Resize((420, 380)), 
                         transforms.ToTensor(), 
                         transforms.Normalize((0.5,), (0.5,))
                         ]))
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle = True)

In [86]:
def RandomImagePrediction(filepath):
    img_array = Image.open(filepath).convert('RGB')
    data_transforms=transforms.Compose([
        transforms.Resize((420, 380)), 
        transforms.ToTensor(), 
        transforms.Normalize((0.5,), (0.5,))
    ])
    img = data_transforms(img_array).unsqueeze(dim=0)
    load = DataLoader(img)
    
    for x in load:
        x=x.to(device)
        pred = model(x)
        _, preds = torch.max(pred, 1)
        return preds

In [87]:
import pandas as pd
left = os.listdir('test_set/left/')
right = os.listdir('test_set/right/')

df_left = pd.DataFrame(left)
df_left['target'] = 0
df_left['path'] = 'test_set/left/'

df_right = pd.DataFrame(right)
df_right['target'] = 1
df_right['path'] = 'test_set/right/'

df = pd.concat([df_left, df_right]).reset_index()

df['path'] = df['path'] + df[0]
df['predict'] = 0

In [88]:
for ix in tqdm(df.index[:]):
    df['predict'][ix] = RandomImagePrediction(df['path'][ix])
    # print(df['predict'][ix])

  0%|          | 0/801 [00:00<?, ?it/s]/home/matthew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 801/801 [01:09<00:00, 11.53it/s]


In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(df['target'], df['predict'])

0.7215980024968789